## Importing libraries and dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
df = pd.read_csv("../data/property.csv")
df

,Unnamed: 0,Price,Category,Beds,Baths,Area,Plot_size,Address,Broker,URL
0,0,"$60,000",Condo for sale,2,2,"1,220sqft\n1,220 square feet",NaN,"10 Harbor Blvd Unit E404D, Destin, FL 32541",Brokered by Legendary Realty Llc,https://www.realtor.com/realestateandhomes-det...
1,1,"$381,000",House for sale,4,3,"1,926sqft\n1,926 square feet",0.6acre lot\n0.6 acre lot,"17561 SW 40 Ct, Ocala, FL 34473",Brokered by Keller Williams Classic Realty,https://www.realtor.com/realestateandhomes-det...
2,2,"$70,000",Condo for sale,2,2,"1,100sqft\n1,100 square feet",NaN,"617 Front /Week 49 St Unit A54, Key West, FL 3...",Brokered by Exit Realty of Naples,https://www.realtor.com/realestateandhomes-det...
3,3,"$9,500",Mobile house for sale,2,2,NaN,NaN,"4020 68th Pl N Unit 827, West Palm Beach, FL 3...",Brokered by American National Homes LLC,https://www.realtor.com/realestateandhomes-det...
4,4,"$1,185,000",Coming Soon,4,3,"2,810sqft\n2,810 square feet",0.5acre lot\n0.5 acre lot,"5307 Solway Dr, Melbourne Beach, FL 32951",Brokered by ONE Sotheby's International Realty,https://www.realtor.com/realestateandhomes-det...
...,...,...,...,...,...,...,...,...,...,...
12687,12687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12688,12688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12689,12689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12690,12690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data Cleaning

### If we clean the rows with null URLs, Rents and null rows will be cleaned easily!

In [147]:
# Remove rows
df = df.dropna(subset=['URL'])

In [148]:
# Function to clean the 'Area' column, get numrical values
def clean_area(area):
    if pd.isna(area):
        return np.nan
    # Extract the first numerical value found in the string
    num_str = ''.join(filter(str.isdigit, area.split()[0]))
    return int(num_str) if num_str else np.nan

df['Area'] = df['Area'].apply(clean_area)

### Cleaning Plot_size text and convert acre to sqft

In [149]:
import re

# Function to convert plot size to square feet
def convert_plot_size(plot_size):
    # Check if plot_size is a string
    if isinstance(plot_size, str):
        # Remove extra spaces
        plot_size = plot_size.strip().lower()

        # If 'acre' is in the plot_size string, convert to sqft
        if 'acre' in plot_size:
            # Extract the numeric value before 'acre'
            acre_value = re.findall(r'[\d.]+', plot_size)
            if acre_value:
                acre_to_sqft = float(acre_value[0]) * 43560  # Convert acre to sqft
                return acre_to_sqft
        
        # If 'sqft' or 'square foot' is in the plot_size string, extract the numeric value directly
        elif 'sqft' in plot_size or 'square foot' in plot_size:
            sqft_value = re.findall(r'[\d.]+', plot_size)
            if sqft_value:
                return float(sqft_value[0])  # Return the numeric value as it is already in sqft
    
    # If plot_size is already a number, return it as is
    elif isinstance(plot_size, (int, float)):
        return plot_size

    # If no relevant unit is found, return None or the original value
    return None

# Apply the function to the Plot_size column
df['Plot_size'] = df['Plot_size'].apply(convert_plot_size)


### Cleaning Broker column

In [150]:
# Function to clean the first two words in the 'Broker' column
def clean_broker_name(broker_name):
    if isinstance(broker_name, str):
        # Split the broker name into words
        words = broker_name.split()
        # Take the first two words and join them back into a string
        cleaned_name = ' '.join(words[2:])
        return cleaned_name
    return broker_name

# Apply the function to the 'Broker' column
df['Broker'] = df['Broker'].apply(clean_broker_name)

In [151]:
# Clean the 'for sale' from the Category
df['Category'] = df['Category'].str.replace('for sale', '', regex=False).str.strip()

### Fixing price data

In [152]:
# Remove non-numeric characters and handle errors
df['Price'] = df['Price'].str.replace('$', '', regex=False).str.replace(',', '')

# Convert to numeric, setting errors='coerce' to handle non-numeric values
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')


### Zip Code Extraction

In [153]:

df['ZIP'] = df['Address'].str.split().str[-1]


In [154]:
import pgeocode
nomi = pgeocode.Nominatim('US')

def get_place_name(zip_code):
    if pd.notna(zip_code):  # Check if ZIP code is not NaN
        location_info = nomi.query_postal_code(zip_code)
        if location_info is not None:
            return location_info.place_name
    return None

# Apply the function to the 'ZIP' column
df['Place_name'] = df['ZIP'].apply(get_place_name)



In [155]:


df['ZIP'] = pd.to_numeric(df['ZIP'], errors='coerce')

### Rename columns

In [156]:
# Rename the columns
df.rename(columns={
    'Price': 'Price($)',
    'Beds': 'Bedrooms',
    'Baths': 'Bathrooms',
    'Area': 'Area(sqft)',
    'Plot_size': 'Plot_size(sqft)'
}, inplace=True)

In [157]:
len(df)

8652

There is 206 pages in the website. Each page contains 42 listing. 206*42 = 8652. So our dataset is ready!!!

In [158]:
df

,Price($),Category,Bedrooms,Bathrooms,Area(sqft),Plot_size(sqft),Address,Broker,URL,ZIP,Place_name
0,60000.0,Condo,2,2,1220.0,NaN,"10 Harbor Blvd Unit E404D, Destin, FL 32541",Legendary Realty Llc,https://www.realtor.com/realestateandhomes-det...,32541.0,Destin
1,381000.0,House,4,3,1926.0,26136.0,"17561 SW 40 Ct, Ocala, FL 34473",Keller Williams Classic Realty,https://www.realtor.com/realestateandhomes-det...,34473.0,Ocala
2,70000.0,Condo,2,2,1100.0,NaN,"617 Front /Week 49 St Unit A54, Key West, FL 3...",Exit Realty of Naples,https://www.realtor.com/realestateandhomes-det...,33040.0,Key West
3,9500.0,Mobile house,2,2,NaN,NaN,"4020 68th Pl N Unit 827, West Palm Beach, FL 3...",American National Homes LLC,https://www.realtor.com/realestateandhomes-det...,33404.0,West Palm Beach
4,1185000.0,Coming Soon,4,3,2810.0,21780.0,"5307 Solway Dr, Melbourne Beach, FL 32951",ONE Sotheby's International Realty,https://www.realtor.com/realestateandhomes-det...,32951.0,Melbourne Beach
...,...,...,...,...,...,...,...,...,...,...,...
12667,104900.0,Mobile house,2,2,1232.0,NaN,"2378 Snowy Plover Dr Unit 12073, Lakeland, FL ...",Cove Communities,https://www.realtor.com/realestateandhomes-det...,33810.0,Lakeland
12668,1200000.0,House,3,1,1128.0,42253.2,NaN,"Starlink Realty, Inc",https://www.realtor.com/realestateandhomes-det...,NaN,None
12669,74500.0,Mobile house,2,2,864.0,NaN,"430 Coquina Dr Unit 1607, Ellenton, FL 34222","Four Star Homes, Inc.",https://www.realtor.com/realestateandhomes-det...,34222.0,Ellenton
12670,399000.0,Pending,3,2,1573.0,NaN,"2365 Bayou Ln Unit 7012, Naples, FL 34112",beycome.com,https://www.realtor.com/realestateandhomes-det...,34112.0,Naples


In [159]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8652 entries, 0 to 12671
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Price($)         8312 non-null   float64
 1   Category         8652 non-null   object 
 2   Bedrooms         7937 non-null   object 
 3   Bathrooms        7900 non-null   object 
 4   Area(sqft)       7461 non-null   float64
 5   Plot_size(sqft)  3613 non-null   float64
 6   Address          8578 non-null   object 
 7   Broker           8646 non-null   object 
 8   URL              8652 non-null   object 
 9   ZIP              8531 non-null   float64
 10  Place_name       8514 non-null   object 
dtypes: float64(4), object(7)
memory usage: 1.0+ MB


In [160]:
# Drop rows where 'Price($)' is null
df = df.dropna(subset=['Price($)'])

In [161]:
df.to_csv('../data/property_cleaned.csv', index=False)